# Coursework 1 - Question 2

## Web References

- [k-means document clustering using Apache Mahout command line](https://datasciencetutos.wordpress.com/2016/08/04/k-means-document-clustering-using-apache-mahout-command-line/)
- [https://bickson.blogspot.com/2011/09/understanding-mahout-k-means-clustering.html](https://bickson.blogspot.com/2011/09/understanding-mahout-k-means-clustering.html)

## Data Preperation

### Upload Data HDFS

In [2]:
%%bash
# change to the coursework directory
cd ~/code/dsm010-2021-oct/coursework_01/

# copy the input documents
 hadoop fs -copyFromLocal data/raw/western_classics/british-fiction-corpus dsm010/british-fiction-corpus

# verify the file uploads
hadoop fs -ls dsm010/british-fiction-corpus

Found 28 items
-rw-r--r--   3 jfoul001 users     378987 2021-12-31 12:37 dsm010/british-fiction-corpus/ABronte_Agnes.txt
-rw-r--r--   3 jfoul001 users     930593 2021-12-31 12:37 dsm010/british-fiction-corpus/ABronte_Tenant.txt
-rw-r--r--   3 jfoul001 users     894304 2021-12-31 12:37 dsm010/british-fiction-corpus/Austen_Emma.txt
-rw-r--r--   3 jfoul001 users     683758 2021-12-31 12:37 dsm010/british-fiction-corpus/Austen_Pride.txt
-rw-r--r--   3 jfoul001 users     678691 2021-12-31 12:37 dsm010/british-fiction-corpus/Austen_Sense.txt
-rw-r--r--   3 jfoul001 users    1026320 2021-12-31 12:37 dsm010/british-fiction-corpus/CBronte_Jane.txt
-rw-r--r--   3 jfoul001 users     506144 2021-12-31 12:37 dsm010/british-fiction-corpus/CBronte_Professor.txt
-rw-r--r--   3 jfoul001 users    1104704 2021-12-31 12:37 dsm010/british-fiction-corpus/CBronte_Villette.txt
-rw-r--r--   3 jfoul001 users    1964819 2021-12-31 12:37 dsm010/british-fiction-corpus/Dickens_Bleak.txt
-rw-r--r--   3 jfoul001 user

### Convert the dataset to SequenceFiles

SequenceFiles are flat files consisting of binary key/value pairs. Each document is represented as a key-value pair. there the key is the document id and value is its content.

In [11]:
%%bash
mahout seqdirectory -i dsm010/british-fiction-corpus -o dsm010/british-fiction-corpus-seqfiles -c UTF-8 -chunk 5

MAHOUT_LOCAL is not set; adding HADOOP_CONF_DIR to classpath.
Running on hadoop, using /opt/hadoop/current/bin/hadoop and HADOOP_CONF_DIR=/opt/hadoop/current/etc/hadoop
MAHOUT-JOB: /opt/mahout/current/mahout-examples-0.13.0-job.jar


21/12/31 14:53:41 INFO AbstractJob: Command line arguments: {--charset=[UTF-8], --chunkSize=[5], --endPhase=[2147483647], --fileFilterClass=[org.apache.mahout.text.PrefixAdditionFilter], --input=[dsm010/british-fiction-corpus], --keyPrefix=[], --method=[mapreduce], --output=[dsm010/british-fiction-corpus-seqfiles], --startPhase=[0], --tempDir=[temp]}
21/12/31 14:53:41 INFO deprecation: mapred.input.dir is deprecated. Instead, use mapreduce.input.fileinputformat.inputdir
21/12/31 14:53:41 INFO deprecation: mapred.compress.map.output is deprecated. Instead, use mapreduce.map.output.compress
21/12/31 14:53:41 INFO deprecation: mapred.output.dir is deprecated. Instead, use mapreduce.output.fileoutputformat.outputdir
21/12/31 14:53:42 INFO DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at lena-master/128.86.245.64:8032
21/12/31 14:53:42 INFO JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/jfoul001/.staging/job_1626049283275_226651
21/12/31

In [3]:
%%bash
hadoop fs -ls dsm010/british-fiction-corpus-seqfiles

Found 8 items
-rw-r--r--   3 jfoul001 users          0 2021-12-31 14:53 dsm010/british-fiction-corpus-seqfiles/_SUCCESS
-rw-r--r--   3 jfoul001 users    2503651 2021-12-31 14:53 dsm010/british-fiction-corpus-seqfiles/part-m-00000
-rw-r--r--   3 jfoul001 users    2277756 2021-12-31 14:53 dsm010/british-fiction-corpus-seqfiles/part-m-00001
-rw-r--r--   3 jfoul001 users    2149725 2021-12-31 14:53 dsm010/british-fiction-corpus-seqfiles/part-m-00002
-rw-r--r--   3 jfoul001 users    2043774 2021-12-31 14:53 dsm010/british-fiction-corpus-seqfiles/part-m-00003
-rw-r--r--   3 jfoul001 users    2026798 2021-12-31 14:53 dsm010/british-fiction-corpus-seqfiles/part-m-00004
-rw-r--r--   3 jfoul001 users    1966802 2021-12-31 14:53 dsm010/british-fiction-corpus-seqfiles/part-m-00005
-rw-r--r--   3 jfoul001 users     559202 2021-12-31 14:53 dsm010/british-fiction-corpus-seqfiles/part-m-00006


### Convert sequenceFiles to sparse vector files

In [5]:
%%bash
mahout seq2sparse -nv -i dsm010/british-fiction-corpus-seqfiles -o dsm010/british-fiction-corpus-vectors

MAHOUT_LOCAL is not set; adding HADOOP_CONF_DIR to classpath.
Running on hadoop, using /opt/hadoop/current/bin/hadoop and HADOOP_CONF_DIR=/opt/hadoop/current/etc/hadoop
MAHOUT-JOB: /opt/mahout/current/mahout-examples-0.13.0-job.jar


21/12/31 15:13:43 INFO SparseVectorsFromSequenceFiles: Maximum n-gram size is: 1
21/12/31 15:13:43 INFO SparseVectorsFromSequenceFiles: Minimum LLR value: 1.0
21/12/31 15:13:43 INFO SparseVectorsFromSequenceFiles: Number of reduce tasks: 1
21/12/31 15:13:43 INFO SparseVectorsFromSequenceFiles: Tokenizing documents in dsm010/british-fiction-corpus-seqfiles
21/12/31 15:13:44 INFO DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at lena-master/128.86.245.64:8032
21/12/31 15:13:44 INFO JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/jfoul001/.staging/job_1626049283275_226808
21/12/31 15:13:44 INFO FileInputFormat: Total input files to process : 7
21/12/31 15:13:44 INFO JobSubmitter: number of splits:7
21/12/31 15:13:44 INFO JobSubmitter: Submitting tokens for job: job_1626049283275_226808
21/12/31 15:13:44 INFO JobSubmitter: Executing with tokens: []
21/12/31 15:13:45 INFO Configuration: resource-types.xml not found
21/12/31 15:13:45 INFO R

In [7]:
%%bash
hadoop fs -ls dsm010/british-fiction-corpus-vectors

Found 7 items
drwxr-xr-x   - jfoul001 users          0 2021-12-31 15:15 dsm010/british-fiction-corpus-vectors/df-count
-rw-r--r--   3 jfoul001 users     800064 2021-12-31 15:14 dsm010/british-fiction-corpus-vectors/dictionary.file-0
-rw-r--r--   3 jfoul001 users     775353 2021-12-31 15:15 dsm010/british-fiction-corpus-vectors/frequency.file-0
drwxr-xr-x   - jfoul001 users          0 2021-12-31 15:15 dsm010/british-fiction-corpus-vectors/tf-vectors
drwxr-xr-x   - jfoul001 users          0 2021-12-31 15:16 dsm010/british-fiction-corpus-vectors/tfidf-vectors
drwxr-xr-x   - jfoul001 users          0 2021-12-31 15:13 dsm010/british-fiction-corpus-vectors/tokenized-documents
drwxr-xr-x   - jfoul001 users          0 2021-12-31 15:14 dsm010/british-fiction-corpus-vectors/wordcount


In [10]:
%%bash
hadoop fs -ls dsm010/british-fiction-corpus-vectors/tf-vectors

Found 2 items
-rw-r--r--   3 jfoul001 users          0 2021-12-31 15:15 dsm010/british-fiction-corpus-vectors/tf-vectors/_SUCCESS
-rw-r--r--   3 jfoul001 users    2680027 2021-12-31 15:15 dsm010/british-fiction-corpus-vectors/tf-vectors/part-r-00000
